# Load the model

In [ ]:
from ludwig.api import LudwigModel

model = LudwigModel.load('results/api_experiment_run_3/model')

predictions, _ = model.predict(dataset='rotten_tomatoes_test.csv')
predictions.head()

# Export the model to Torchscript

https://ludwig.ai/latest/user_guide/model_export/#torchscript-export

In [26]:
!ludwig export_torchscript -m=results/api_experiment_run_11/model

███████████████████████
█ █ █ █  ▜█ █ █ █ █   █
█ █ █ █ █ █ █ █ █ █ ███
█ █   █ █ █ █ █ █ █ ▌ █
█ █████ █ █ █ █ █ █ █ █
█     █  ▟█     █ █   █
███████████████████████
ludwig v0.7.dev - Export Torchscript

Model path: results/api_experiment_run_11/model
Saving model only: False
output_path is None, defaulting to model_path
Output path: results/api_experiment_run_11/model


/usr/local/lib/python3.8/site-packages/sklearn/experimental/enable_hist_gradient_boosting.py:16: UserWarning: Since version 1.0, it is not needed to import enable_hist_gradient_boosting anymore. HistGradientBoostingClassifier and HistGradientBoostingRegressor are now stable and can be normally imported from sklearn.ensemble.
  warnings.warn(
/workspaces/ludwig/ludwig/models/gbm.py:135: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might no

# Use Torchscript to run model inference

In [28]:
import torch


preprocessor = torch.jit.load("results/api_experiment_run_11/model/inference_preprocessor.pt")
predictor = torch.jit.load("results/api_experiment_run_11/model/inference_predictor-cpu.pt")
postprocessor = torch.jit.load("results/api_experiment_run_11/model/inference_postprocessor.pt")

In [29]:
from ludwig.utils.inference_utils import to_inference_module_input_from_dataframe
import pandas as pd
import yaml


test_df = pd.read_csv('rotten_tomatoes_test.csv')
raw_data = to_inference_module_input_from_dataframe(test_df, config=model.config)

preprocessed_data = preprocessor(raw_data)
predictions = predictor(preprocessed_data)
postprocessed_data = postprocessor(predictions)

print(postprocessed_data)

{'recommended::predictions': tensor([ True,  True,  True,  True,  True,  True,  True,  True,  True, False]), 'recommended::probabilities': tensor([[0.2415, 0.7585],
        [0.3624, 0.6376],
        [0.1973, 0.8027],
        [0.3849, 0.6151],
        [0.2594, 0.7406],
        [0.2294, 0.7706],
        [0.3130, 0.6870],
        [0.2874, 0.7126],
        [0.4713, 0.5287],
        [0.5144, 0.4856]])}
